In [ ]:
!git clone https://github.com/dna-witch/KGPL-PyTorch/ -b th-dev

Cloning into 'KGPL-PyTorch'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 292 (delta 65), reused 11 (delta 11), pack-reused 169 (from 1)
Receiving objects: 100% (292/292), 7.49 MiB | 24.57 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [ ]:
!pip install -r KGPL-PyTorch/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 4.3 MB/s eta 0:00:00


In [ ]:
!cd KGPL-PyTorch &&\
python preprocess/preprocess.py -d "music" &&\
python preprocess/make_path_list.py lp_depth=6 dataset=music kg_path=data/music/kg_final.npy rating_path=data/music/ratings_final.npy num_neighbor_samples=32

reading item index to entity id file: data/music/item_index2entity_id.txt ...
reading rating file ...
converting rating file ...
number of users: 1872
number of items: 3846
converting kg file ...
number of entities (containing items): 9366
number of relations: 60
done
adj_entity_path: data/music/adj_entity_6_32.npy
adj_relation_path: data/music/adj_relation_6_32.npy
data_path: data/music/fold1.pkl
dataset: music
kg_path: data/music/kg_final.npy
lp_depth: 6
num_neighbor_samples: 32
pathlist_path: data/music/path_list_6_32.pkl
rating_path: data/music/ratings_final.npy
reachable_items_path: data/music/reachable_items.pkl

[Parallel(n_jobs=32)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Batch computation too fast (0.02620244026184082s.) Setting batch_size=2.
[Parallel(n_jobs=32)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=32)]: Done  21 tasks      | elapsed:    0.4s
[Parallel(n_jobs=32)]: Done  34 tasks      | elapsed:    0.6s
[Pa

In [ ]:
import sys
sys.path.append('KGPL-PyTorch/')

In [ ]:
from model import *
from utils.data_classes import *
from utils.functions import *
from utils.eval import *

import torch
from tqdm import tqdm

In [ ]:
music = KGPLExperiment(
    'music',
    {
      'plabel_lp_depth': 6,
      'plabel_par': 16,
      'plabel_chunk_size': 250,
      'plabel_neg_pn': 0.1,
      'plabel_pl_pn': 1e-3,
      'dropout_rate': 0.5,
      'emb_dim': 64,
      'n_iter': 1,
      'plabel': {},
      'optimize': {'iter_per_epoch': 100, 'lr': 3e-3, 'batch_size': 3333},
      'log': {'show_loss': True},
      'evaluate': {'user_num_topk': 1000},
      'model': {
          'n_iter': 1,
          'neighbor_sample_size': 32,
          'num_neighbor_samples': 32,
          'dropout_rate': 0.5}
    })

Loading Entity Adjacencies...
Loading Relation Adjacencies...
Loading Ratings...
Loading Path List...
Loading Distances...
Setting up dst dict...


100%|██████████| 3846/3846 [00:00<00:00, 11951.27it/s]

Start updating path info...
Path info updated.


In [ ]:
music.train_val_test_split()

/content/KGPL-PyTorch/utils/data_classes.py:197: RuntimeWarning: invalid value encountered in divide
  F = (F / F.sum()).cumsum()


Neg C Dict User: 1870
Neg C Dict Item: 3829


100%|██████████| 27098/27098 [00:00<00:00, 203219.93it/s]


Populating item dist dict...


8it [00:03,  2.51it/s]


In [ ]:
music.create_dataloaders()

In [ ]:
model = KGPLCOT(
        music.cfg,
        int(music.ratings[:, 0].max().item()) + 1,
        int(music.ratings[:, 1].max().item()) + 1,
        music.n_entity,
        music.n_relation,
        music.adj_entity,
        music.adj_relation,
        music.path_list_dict,
        music.train_loader,
        music.val_loader,
        'cpu'
    )

In [ ]:
def run_cotrain(model, train_loader, val_loader, test_loader, cfg, device, num_epochs=40, k=10):
    optimizer_f = torch.optim.Adam(model.model_f.parameters(), lr=cfg['optimize']['lr'])
    optimizer_g = torch.optim.Adam(model.model_g.parameters(), lr=cfg['optimize']['lr'])
    iter_per_epoch = cfg['optimize']['iter_per_epoch']

    for epoch in range(1, num_epochs + 1):
        print(f"\nEpoch {epoch}")
        model.train()
        total_loss_f, total_loss_g = 0.0, 0.0

        train_iter = iter(train_loader)  # re-initialize iterator every epoch

        for i in tqdm(range(iter_per_epoch)):
            try:
                batch = next(train_iter)
            except StopIteration:
                train_iter = iter(train_loader)
                batch = next(train_iter)

            users, pos, neg, pseudo = [b.to(device) for b in batch]
            batch_data = torch.stack([users, pos, neg, pseudo], dim=1)

            losses = model.train_step(batch_data, optimizer_f, optimizer_g)
            total_loss_f += losses['loss_f']
            total_loss_g += losses['loss_g']

        avg_loss_f = total_loss_f / iter_per_epoch
        avg_loss_g = total_loss_g / iter_per_epoch
        print(f"Train Loss - f: {avg_loss_f:.4f}, g: {avg_loss_g:.4f}")

        print("Evaluating model_f:")

        run_topk_eval(
          model=model,
          cfg = cfg,
          train_data=music.train_dataset.ratings.numpy(),
          eval_data=music.val_dataset.ratings.numpy(),
          test_data=music.test_dataset.ratings.numpy(),
          n_items=music.n_item,
          device=device,
          test_mode=False  # or True for test set
        )

In [ ]:
run_cotrain(
    model,
    music.train_loader,
    music.val_loader,
    music.test_loader,
    music.cfg,
    'cpu'
)


Epoch 1


100%|██████████| 100/100 [01:13<00:00,  1.35it/s]


Train Loss - f: 5.0054, g: 4.9993
Evaluating model_f:


Evaluating: 100%|██████████| 1000/1000 [00:14<00:00, 70.67it/s]



Top-K Evaluation Results
K=  1 | Precision: 0.0000 | Recall: 0.0000
K=  2 | Precision: 0.0005 | Recall: 0.0005
K=  5 | Precision: 0.0006 | Recall: 0.0010
K= 10 | Precision: 0.0004 | Recall: 0.0015
K= 20 | Precision: 0.0004 | Recall: 0.0042

Epoch 2


...

100%|██████████| 100/100 [01:08<00:00,  1.46it/s]


Train Loss - f: 1.8696, g: 1.8613
Evaluating model_f:


Evaluating: 100%|██████████| 1000/1000 [00:19<00:00, 51.36it/s]



Top-K Evaluation Results
K=  1 | Precision: 0.0250 | Recall: 0.0105
K=  2 | Precision: 0.0270 | Recall: 0.0247
K=  5 | Precision: 0.0258 | Recall: 0.0624
K= 10 | Precision: 0.0221 | Recall: 0.1061
K= 20 | Precision: 0.0171 | Recall: 0.1719

Epoch 30


 25%|██▌       | 25/100 [00:17<00:51,  1.45it/s]

In [ ]:
|